In [105]:
import pandas
import os
import re

In [106]:
SAMPLE = 'sample_0'
SOLVED_FILE_PATH = 'data/output/solved_2/'
EQUATIONS_FILE_PATH = ''

In [107]:
ground = pandas.read_json('data/input/draw.json')
ground = ground[['lSolutions', 'iIndex']]
ground = ground.set_index('iIndex')
display(len(ground))
ground.head(1)

999

,lSolutions
iIndex,
397760,[2.14285714286]


In [108]:
# solved_majority = pandas.read_json('data/output/majority_solved/majority_solved.jsonl', lines=True)
# solved_majority.head(1)

In [109]:
def extract_actual_answer(answer):
    answer = re.sub(r'\s', "", answer)
    answer = re.sub(r'\s|.*####', "", answer)
    return eval(answer)

def remove_leading_zero(number : str):
    return re.sub(r'^0+', '', number)

def extract_chatgpt_answer(answer_text):
    answer_text = answer_text.lower()
    try:
        answers = re.findall(r'the\s*answer\s*is\s*.*', answer_text)[-1]
        numbers = re.findall(r'\d+\.?\d*', answers)
        fractions = re.findall(r'\d+\/\d+', answers)
        answers = numbers + fractions

        to_return = []
        for answer in answers:
            try: to_return.append(round(eval(remove_leading_zero(answer))))
            except: pass
        return set(to_return)
    except:
        answers = re.findall(r'\d+\.?\d*', answer_text)
        if len(answers) == 0: return set()
        return set([eval(answers[0])])
    
def extract_solved(solved):
    answers = re.findall(r'\d+\.?\d*', solved)
    return set([round(eval(answer), 2) for answer in answers])

def grade_response(row, file_name):
    chatgpt_ans = row['majority_answer']    #NOEL HELP ------------------------- USE THE MAJORITY_ANSWER THING I SET UP.
    solved = row[f'{file_name}__solved']
    system = row['choices']
    solved = extract_solved(solved)
    system = extract_chatgpt_answer(system)
    actual_answer = set([str(round(float(s), 2)) for s in row['lSolutions']])

    chatgpt_ans = set([str(round(float(s), 2)) for s in chatgpt_ans])    #NOEL HELP ------------------------- USE THE MAJORITY_ANSWER THING I SET UP.
    solved = set([str(round(float(s), 2)) for s in solved])
    system = set([str(round(float(s), 2)) for s in system])
    actual_answer = set([str(round(float(s), 2)) for s in actual_answer])
    
    row['lSolutions'] = actual_answer
    row[f'chatgpt_answer'] = chatgpt_ans
    row[f'solved_answer'] = solved
    row[f'system_answer'] = system

    A = 1 if chatgpt_ans.issubset(actual_answer) and len(chatgpt_ans) > 0 else 0
    # A = 1 if (chatgpt_answer.issubset(actual_answer) or actual_answer.issubset(chatgpt_answer)) and len(actual_answer.intersection(chatgpt_answer)) > 0 else 0
    # A = 1 if chatgpt_answer.intersection(actual_answer) else 0

    C = 1 if system.issubset(actual_answer) and len(system) > 0 else 0
    D = 1 if solved.issubset(actual_answer) and len(solved) > 0 else 0

    # A = 1 if len(chatgpt_answer.intersection(actual_answer)) > 0 else 0
    # C = 1 if len(system.intersection(actual_answer)) > 0 else 0
    # D = 1 if len(solved.intersection(actual_answer)) > 0 else 0

    row['A. ChatGPT gets it correct from text to final solution'] = A
    row['B. Sympy gets it correct from equations to final solution'] = 1
    row['C. ChatGPT gets it correct from equations to final solution'] = C
    row['D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT'] = D

    category_array = [A, C, D]

    equation_final_same = (chatgpt_ans.issubset(solved) or solved.issubset(chatgpt_ans)) and len(solved.intersection(chatgpt_ans)) > 0

    offset = 8 if equation_final_same else 0
    row['E. Final answer and equations same'] = 1 if equation_final_same else 0
    # row['E. Final answer and equations same'] = 1 if len(chatgpt_answer.intersection(solved)) > 0 else 0
    
    if category_array == [0, 0 , 0]: row['category'] = 1 + offset
    if category_array == [0, 0 , 1]: row['category'] = 2 + offset
    if category_array == [0, 1 , 0]: row['category'] = 3 + offset
    if category_array == [0, 1 , 1]: row['category'] = 4 + offset
    if category_array == [1, 0 , 0]: row['category'] = 5 + offset
    if category_array == [1, 0 , 1]: row['category'] = 6 + offset
    if category_array == [1, 1 , 0]: row['category'] = 7 + offset
    if category_array == [1, 1 , 1]: row['category'] = 8 + offset
    
    return row


In [110]:
system = pandas.read_json('data/output/system_equations/sample_0.jsonl', lines=True)
system = system[['question_id', 'choices']]
system['choices'] = system['choices'].apply(lambda row : row[0]['message']['content'])
system = system.set_index('question_id')
system.head(1)

,choices
question_id,
74243,There seems to be only one equation given. Can...


In [111]:
majority_ans = pandas.read_json('data/output/majority_2/sample_0.jsonl', lines=True)
majority_ans = majority_ans[['question_id', 'majority_answer']]
majority_ans = majority_ans.set_index('question_id')
ground = ground.join(majority_ans)
ground.head(5)

,lSolutions,majority_answer
337,[2.4],[2.4]
1042,"[9.0, 12.0]","[9, 12]"
1299,"[12.5, 87.5]","[12.5, 87.5]"
1422,[25.0],[25]
1431,"[4.0, 6.0]","[4, 6]"


In [112]:
ground = ground.join(system)

for file_path in os.listdir(SOLVED_FILE_PATH):
    solved = pandas.read_json(f"{SOLVED_FILE_PATH}{file_path}", lines=True)
    solved = solved[['question_id', 'question', 'solved']]
    # solved['choices'] = solved['choices'].apply(lambda row : row[0]['message']['content'])
    file_name = file_path.split('.')[0]
    print("here: ", str(file_name))
    solved = solved.rename(columns={'question': f'{file_name}__question', 'solved': f'{file_name}__solved'})
    solved = solved.set_index('question_id')
    ground = ground.join(solved, how='left')
    ground = ground.apply(lambda row : grade_response(row, file_name), axis=1)

# majority_eqn = pandas.read_json('data/output/solve_majority_test/sample_0.jsonl',lines=True)
majority_eqn = pandas.read_json('data/output/majority/sample_0.jsonl',lines=True)
# majority_eqn = majority_eqn[['question_id', 'response', 'majority_equation', 'solved']]
majority_eqn = majority_eqn[['question_id', 'majority_equation', 'majoirty_solved']]
majority_eqn = majority_eqn.rename(columns={'question': f'majority__question', 'majority_equation': f'majority__choices', 'majority_solved': f'majority__solved'})
majority_eqn = majority_eqn.set_index('question_id')
ground = ground.join(majority_eqn)
ground = ground.apply(lambda row : grade_response(row, 'majority'), axis=1)

ground = ground[~ground.index.duplicated(keep='first')]
ground.head(3)

here:  sample_0
here:  sample_1
here:  sample_2
here:  sample_3
here:  sample_4
here:  sample_5
here:  sample_6
here:  sample_7
here:  sample_8
here:  sample_9


KeyError: 'majority__solved'

In [ ]:
ground.to_json('data/output/joined/sample_0.jsonl', orient='records', lines=True)
ground.to_csv('data/output/joined/sample_0.csv')

In [ ]:
def label_category(row):
    category = row.name[0]
    if category > 8:
        row['E. Final answer and equations same'] = 1
        category -= 8
    else:
        row['E. Final answer and equations same'] = 0

    if category == 1:
        row['Equations'] = '''UNKNOWN'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Definitely cannot solve, may have got wrong rep.'''
        row['Understandable'] = '''YES'''
    elif category == 2:
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Gets equations right, but solves wrong'''
        row['Understandable'] = '''YES'''
    elif category == 3:
        row['Equations'] = '''FAIL'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Gets equations wrong, but it can solve otherwise.'''
        row['Understandable'] = '''YES'''
    elif category == 4:
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Non-determinism issues as we do not know why it can't handle by itself'''
        row['Understandable'] = '''NO'''
    elif category == 5:
        row['Equations'] = '''UNKNOWN'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Non-determinism as we do not know how it could gotten the problem right'''
        row['Understandable'] = '''NO'''
    elif category == 6:
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Can get the right equations, perhaps there is something about the equations from ground truth that mess it up'''
        row['Understandable'] = '''MAYBE'''
    elif category == 7:
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''ChatGPT is lying on the equations it reports'''
        row['Understandable'] = '''MAYBE'''
    elif category == 8:
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''SUCCEED'''
        row['Theory'] = '''Gets everything right'''
        row['Understandable'] = '''YES'''
    return row


values = pandas.DataFrame(ground[[
    'category',
    'A. ChatGPT gets it correct from text to final solution',
    'B. Sympy gets it correct from equations to final solution',
    'C. ChatGPT gets it correct from equations to final solution',
    'D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT',
]].value_counts().sort_index())

# # TODO: Remove this abomination
# join = values.copy()
# # display(values.index)
# # display(join.index)
# join.loc[1, 0, 1, 0, 0] = {'count': 0}
# join.loc[2, 0, 1, 0, 1] = {'count': 0}
# join.loc[3, 0, 1, 1, 0] = {'count': 0}
# join.loc[4, 0, 1, 1, 1] = {'count': 0}
# join.loc[5, 1, 1, 0, 0] = {'count': 0}
# join.loc[6, 1, 1, 0, 1] = {'count': 0}
# join.loc[7, 1, 1, 1, 0] = {'count': 0}
# join.loc[8, 1, 1, 1, 1] = {'count': 0}

# join.loc[9, 0, 1, 0, 0] = {'count': 0}
# join.loc[10, 0, 1, 0, 1] = {'count': 0}
# join.loc[11, 0, 1, 1, 0] = {'count': 0}
# join.loc[12, 0, 1, 1, 1] = {'count': 0}
# join.loc[13, 1, 1, 0, 0] = {'count': 0}
# join.loc[14, 1, 1, 0, 1] = {'count': 0}
# join.loc[15, 1, 1, 1, 0] = {'count': 0}
# join.loc[16, 1, 1, 1, 1] = {'count': 0}
# join.index.names = values.index.names
# # display(join)

# # display(values.index)
# # display(join.index)
# values = values.join(join, how='right', rsuffix='__right', lsuffix='__left')
# values['count__left'] = values['count__left'].fillna(0)
# values = values.rename(columns={'count__left': 'count'})
# values = values.drop(columns=['count__right'])

# values = values.apply(lambda row: label_category(row.sort_index()), axis=1)
# values = values.sort_index()
# values = values[['E. Final answer and equations same', 'count', 'Equations', 'Solving', 'Theory', 'Understandable']]
values

,,,,,0
category,A. ChatGPT gets it correct from text to final solution,B. Sympy gets it correct from equations to final solution,C. ChatGPT gets it correct from equations to final solution,D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT,
1,0,1,0,0,52
2,0,1,0,1,21
3,0,1,1,0,28
4,0,1,1,1,38
5,1,1,0,0,82
6,1,1,0,1,1
7,1,1,1,0,45
8,1,1,1,1,5
9,0,1,0,0,43


In [ ]:
writer = pandas.ExcelWriter('data/output/xlsx/Categories.xlsx', engine='xlsxwriter')
values.to_excel(writer, sheet_name='List')

workbook = writer.book

wrap_format = workbook.add_format()
wrap_format.set_text_wrap()

list_sheet = writer.sheets['List']
list_sheet.set_column('A:H', 15, wrap_format)

# When equations = text to final
pie_chart = workbook.add_chart({'type': 'pie'})
pie_chart.set_legend({'position': 'overlay_right'})
pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'When equations != text to final'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=List!$A$2:$A$9",
    'values': f"=List!$G$2:$G$9",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}},
        {'fill': {'color': '#ffc619'}},
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('A20', pie_chart, {'x_offset': 0, 'y_offset': 0})

# When equations != final-to-text
pie_chart = workbook.add_chart({'type': 'pie'})
pie_chart.set_legend({'position': 'overlay_right'})
pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'When equations = text to final'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=List!$A$10:$A$17",
    'values': f"=List!$G$10:$G$17",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}},
        {'fill': {'color': '#ffc619'}},
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('H20', pie_chart, {'x_offset': 0, 'y_offset': 0})

list_sheet.write(49, 0, 'When equations != text to final')
list_sheet.write(50, 0, 'Wrong')
list_sheet.write(51, 0, 'Correct')
list_sheet.write(50, 1, values[values.apply(lambda row: row['E. Final answer and equations same'] == 0 and row.name[1] == 0, axis=1)]['count'].sum())
list_sheet.write(51, 1, values[values.apply(lambda row: row['E. Final answer and equations same'] == 0 and row.name[1] == 1, axis=1)]['count'].sum())

# Wrong - Right ratio when equations != text to final
pie_chart = workbook.add_chart({'type': 'pie'})
pie_chart.set_legend({'position': 'overlay_right'})
pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'When equations != text to final'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=List!$A$51:$A$52",
    'values': f"=List!$B$51:$B$52",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}}
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('A53', pie_chart, {'x_offset': 0, 'y_offset': 0})

list_sheet.write(49, 7, 'When equations = text to final')
list_sheet.write(50, 7, 'Wrong')
list_sheet.write(51, 7, 'Correct')
list_sheet.write(50, 8, values[values.apply(lambda row: row['E. Final answer and equations same'] == 1 and row.name[1] == 0, axis=1)]['count'].sum())
list_sheet.write(51, 8, values[values.apply(lambda row: row['E. Final answer and equations same'] == 1 and row.name[1] == 1, axis=1)]['count'].sum())

pie_chart = workbook.add_chart({'type': 'pie'})
pie_chart.set_legend({'position': 'overlay_right'})
pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'When equations = text to final'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=List!$H$51:$H$52",
    'values': f"=List!$I$51:$I$52",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}}
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('H53', pie_chart, {'x_offset': 0, 'y_offset': 0})

writer.close()

KeyError: 'E. Final answer and equations same'